In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.show_dimensions", False)
pd.set_option("display.float_format", "{:4.2g}".format)

In [5]:
from IPython.core.magic import register_line_magic

@register_line_magic
def C(line):
    from IPython.core.getipython import get_ipython
    from fnmatch import fnmatch

    line = line.strip()
    if ' ' in line:
        idx_space = line.index(' ')
        space_num = line[:idx_space]
        if space_num.isdecimal():
            space_num = int(space_num)
            line = line[idx_space:]
        else:
            space_num = 5
    else:
        space_num = 5

    output_dict = {}
    cmds = line.split(';')
    for cmd in cmds:
        cmd = cmd.strip()
        output_dict[cmd] = repr(eval(cmd)).split("\n")

    str_maxlen_in_cols = [max(len(cmd), len(max(data, key=len))) for cmd, data in output_dict.items()]
    data_row_max = max([len(v) for v in output_dict.values()])

    out_lines = [""]*(data_row_max+2)

    space=''
    for i, (cmd, data) in enumerate(output_dict.items()):
        w = str_maxlen_in_cols[i]

        out_lines[0]+=space+f'{cmd:^{w}}'
        out_lines[1]+=space+"-"*w
        for j, d in enumerate(data, 2):
            out_lines[j]+=space+f'{d:{w}}'

        if len(data) < data_row_max:
            for j in range(len(data)+2, data_row_max+2):
                out_lines[j]+=space+' '*w
        
        space = ' '*space_num

    for line in out_lines:
        print(line)

### 改變DataFrame的形狀

表 5-4 本節要介紹的函數
|函數名稱 |功能 |函數名稱 |功能 |
|--------|-----|--------|-----|
|concat |連接多塊資料 |drop |刪除行或列 |
|set_index |設定索引 |reset_index |將行索引轉為列 |
|stack |將列索引轉為行索引 |unstack |將行索引轉為列索引 |
|reorder_levels |設定索引等級的順序 |swaplevel |交換索引中兩個等級的順序 |
|sort_index |對索引排序 |pivot |建立透視表 |
|melt |透視表的逆轉換 |assign |傳回增加新列之後的資料 |

`DataFrame` 的 `shape` 屬性和 `Numpy` 的二維陣列相同，是一個有兩個元素的元組。由於 `DataFrame` 的 `index` 和 `columns` 都支援 `MultiIndex` 索引物件，因此可以用 `DataFrame` 表示更高維的資料。

下面首先從 CSV 檔案讀取資料，並使用 `groupby()` 計算分組的平均值。關於 `groupby` 在後面的章節還會詳細介紹。注意下面的 `soils_mean` 物件的行索引是多級索引：

In [8]:
soils = pd.read_csv("Soils.csv", index_col=0)[["Depth", "Contour", "Group", "pH", "N"]]
soils_mean = soils.groupby(["Depth", "Contour"]).mean()
%C soils.head(); soils_mean.head()

          soils.head()                        soils_mean.head()        
---------------------------------     ---------------------------------
   Depth Contour  Group   pH    N                       Group   pH    N
1   0-10     Top      1  5.4 0.19     Depth Contour                    
2   0-10     Top      1  5.7 0.17     0-10  Depression      9  5.4 0.18
3   0-10     Top      1  5.1 0.26           Slope           5  5.5 0.22
4   0-10     Top      1  5.1 0.17           Top             1  5.3  0.2
5  10-30     Top      2  5.1 0.16     10-30 Depression     10  4.9 0.08
                                            Slope           6  5.3  0.1


#### 1) 加入移除列或行

由於 `DataFrame` 可以看作一個 `Series` 物件的字典，因此透過 `DataFrame[colname] = values` 即可增加新列。有時新增加的列是從己經存在的列計算而來，這時可以使用 `eval()` 方法計算。例如下面的程式增加一個名為 N_percent 的新列，其值為 N 列乘上 100：

In [9]:
%C soils.head()
soils["N_percent"] = soils.eval("N * 100")
%C soils.head()

          soils.head()           
---------------------------------
   Depth Contour  Group   pH    N
1   0-10     Top      1  5.4 0.19
2   0-10     Top      1  5.7 0.17
3   0-10     Top      1  5.1 0.26
4   0-10     Top      1  5.1 0.17
5  10-30     Top      2  5.1 0.16
                soils.head()                
--------------------------------------------
   Depth Contour  Group   pH    N  N_percent
1   0-10     Top      1  5.4 0.19         19
2   0-10     Top      1  5.7 0.17         16
3   0-10     Top      1  5.1 0.26         26
4   0-10     Top      1  5.1 0.17         17
5  10-30     Top      2  5.1 0.16         16


`assign()` 方法增加由關鍵字參數指定的列，它傳回一個新的 `DataFrame` 物件，原資料的內容保持不變：

In [10]:
print( soils.assign(pH2 = soils.pH + 1).head() )

   Depth Contour  Group   pH    N  N_percent  pH2
1   0-10     Top      1  5.4 0.19         19  6.4
2   0-10     Top      1  5.7 0.17         16  6.7
3   0-10     Top      1  5.1 0.26         26  6.1
4   0-10     Top      1  5.1 0.17         17  6.1
5  10-30     Top      2  5.1 0.16         16  6.1


`append()` 方法用於增加行(row)，它沒有 `inplace` 參數，只能傳回一個全新物件。

由於每次呼叫 `append()` 都會複製所有的資料，因此在循環中使用 `append()` 增加資料會相當大地降低程式的運算速度。可以使用一個列表快取所有的分段資料，然後呼叫 `concat()` 將所有這些資料沿著指定軸拼貼到一起。下面的程式比較二者的運算速度：

In [11]:
def random_dataframe(n):
    columns = ["A", "B", "C"]
    for i in range(n):
        nrow = np.random.randint(10, 20)
        yield pd.DataFrame(np.random.randint(0, 100, size=(nrow, 3)), columns=columns)

df_list = list(random_dataframe(1000))

In [12]:
%%time
df_res1 = pd.DataFrame([])
for df in df_list:
    df_res1 = df_res1.append(df)

Wall time: 245 ms


In [13]:
%%time
df_res2 = pd.concat(df_list, axis=0)

Wall time: 56.5 ms


可以使用 `keys` 參數指定與每塊資料對應的鍵，這樣結果中的連接軸將使用多級索引，方便快速取得原始的資料區塊。

下面取得連接之後的 `DataFrame` 物件中第 0 級標籤為 30 的資料，並使用 `equals()` 方法判斷它是否與原始資料中索引為 30 的資料區塊相同：

In [15]:
df_res3 = pd.concat(df_list, axis=0, keys=range(len(df_list)))
%C df_res2.head(20); df_res3.head(20)
df_res3.loc[30].equals(df_list[30])

df_res2.head(20)     df_res3.head(20)
----------------     ----------------
     A   B   C              A   B   C
0   20  40  35       0 0   20  40  35
1   98  97  83         1   98  97  83
2   74  52  94         2   74  52  94
3   49  16  14         3   49  16  14
4   22  52  97         4   22  52  97
5   21  89  19         5   21  89  19
6    2  12  18         6    2  12  18
7   37  13  51         7   37  13  51
8   57  70  57         8   57  70  57
9   30  11  30         9   30  11  30
10  98  50  21         10  98  50  21
11  62  93  25         11  62  93  25
12  98  17  92         12  98  17  92
13  71  90  73         13  71  90  73
14   3  18  81         14   3  18  81
15  85  49  86         15  85  49  86
16  47  58  82         16  47  58  82
17  77  33  41         17  77  33  41
18  82  34  41         18  82  34  41
0   26  86  37       1 0   26  86  37


True

`drop()` 刪除指定標籤對應的行或列，下面刪除名為 N 和 Group 的兩列：

In [16]:
print( soils.drop(["N", "Group"], axis=1).head() )

   Depth Contour   pH  N_percent
1   0-10     Top  5.4         19
2   0-10     Top  5.7         16
3   0-10     Top  5.1         26
4   0-10     Top  5.1         17
5  10-30     Top  5.1         16


#### 2) 行索引與列之間相互轉換

`reset_index()` 可以將索引轉為列，透過 `level` 參數可以指定被轉為列的等級。如果只希望從索引中刪除某個等級，可以設定 `drop` 參數為 `True`。

In [17]:
print( soils_mean.reset_index(level="Contour").head() )

          Contour  Group   pH    N
Depth                             
0-10   Depression      9  5.4 0.18
0-10        Slope      5  5.5 0.22
0-10          Top      1  5.3  0.2
10-30  Depression     10  4.9 0.08
10-30       Slope      6  5.3  0.1


`set_index()` 將列轉為行索引，如果 `append` 參數為 `False`(預設值)，則刪除目前的行索引；若為 `True`，則為目前的索引增加新的等級。

In [18]:
print( soils_mean.set_index("Group", append=True).head() )

                         pH    N
Depth Contour    Group          
0-10  Depression 9      5.4 0.18
      Slope      5      5.5 0.22
      Top        1      5.3  0.2
10-30 Depression 10     4.9 0.08
      Slope      6      5.3  0.1


#### 3) 行和列的索引相互轉換

`stack()` 方法把指定等級的 列索引 轉為 行索引，而 `unstack()` 則把 行索引 轉為 列索引。

下面的程式將 行索引 中的第一級轉為 列索引 的第一級，所得到的結果中 行索引 為 單級索引，而 列索引 為 多級索引。

In [19]:
%C soils_mean.head()
print( soils_mean.unstack(1)[["Group", "pH"]].head() )

        soils_mean.head()        
---------------------------------
                  Group   pH    N
Depth Contour                    
0-10  Depression      9  5.4 0.18
      Slope           5  5.5 0.22
      Top             1  5.3  0.2
10-30 Depression     10  4.9 0.08
      Slope           6  5.3  0.1
             Group                   pH           
Contour Depression Slope Top Depression Slope  Top
Depth                                             
0-10             9     5   1        5.4   5.5  5.3
10-30           10     6   2        4.9   5.3  4.8
30-60           11     7   3        4.4   4.3  4.2
60-90           12     8   4        4.2   3.9  3.9


無論是 `stack()` 還是 `unstack()` ，當所有的索引被轉換到同一個軸上時，將獲得一個 `Series` 物件：

In [20]:
print( soils_mean.stack().head(10) )

Depth  Contour          
0-10   Depression  Group      9
                   pH       5.4
                   N       0.18
       Slope       Group      5
                   pH       5.5
                   N       0.22
       Top         Group      1
                   pH       5.3
                   N        0.2
10-30  Depression  Group     10
dtype: float64


#### 4) 交換索引的等級

`reorder_levels()` 和 `swaplevel()` 交換指定軸的索引等級。下面呼叫 `swaplevel()` 交換行索引的兩個等級，然後呼叫 `sort_index()` 對新的索引進行排序：

In [21]:
print( soils_mean.swaplevel(0, 1).sort_index() )

                  Group   pH     N
Contour    Depth                  
Depression 0-10       9  5.4  0.18
           10-30     10  4.9  0.08
           30-60     11  4.4 0.051
           60-90     12  4.2  0.04
Slope      0-10       5  5.5  0.22
           10-30      6  5.3   0.1
           30-60      7  4.3 0.061
           60-90      8  3.9 0.043
Top        0-10       1  5.3   0.2
           10-30      2  4.8  0.12
           30-60      3  4.2  0.08
           60-90      4  3.9 0.058


#### 5) 透視表

`pivot()` 可以將 `DataFrame` 中的三列資料分別作為行索引、列索引 和 元素值，將這三列資料轉為二維表格：

In [22]:
df = soils_mean.reset_index()[["Depth", "Contour", "pH", "N"]]
df_pivot_pH = df.pivot("Depth", "Contour", "pH")
%C df; df_pivot_pH

               df                              df_pivot_pH          
--------------------------------     -------------------------------
    Depth     Contour   pH     N     Contour  Depression  Slope  Top
0    0-10  Depression  5.4  0.18     Depth                          
1    0-10       Slope  5.5  0.22     0-10            5.4    5.5  5.3
2    0-10         Top  5.3   0.2     10-30           4.9    5.3  4.8
3   10-30  Depression  4.9  0.08     30-60           4.4    4.3  4.2
4   10-30       Slope  5.3   0.1     60-90           4.2    3.9  3.9
5   10-30         Top  4.8  0.12                                    
6   30-60  Depression  4.4 0.051                                    
7   30-60       Slope  4.3 0.061                                    
8   30-60         Top  4.2  0.08                                    
9   60-90  Depression  4.2  0.04                                    
10  60-90       Slope  3.9 0.043                                    
11  60-90         Top  3.9 0.058  

`pivot()` 的三個參數 `index`, `columns`, `values` 只支援指定 一列資料。若不指定 `values` 參數，就將剩餘的列都當作元素值列，獲得多級列索引：

In [23]:
print( df.pivot("Depth", "Contour") )

                pH                     N            
Contour Depression Slope  Top Depression Slope   Top
Depth                                               
0-10           5.4   5.5  5.3       0.18  0.22   0.2
10-30          4.9   5.3  4.8       0.08   0.1  0.12
30-60          4.4   4.3  4.2      0.051 0.061  0.08
60-90          4.2   3.9  3.9       0.04 0.043 0.058


`melt()` 可以看作 `pivot()` 的逆轉換。由於它不能對行索引操作，因此先呼叫 `reset_index()` 將行索引轉為列，然後用 `id_vars` 參數指定該列為識別欄位：

In [24]:
df_before_melt = df_pivot_pH.reset_index()
df_after_melt = pd.melt(df_before_melt, id_vars="Depth", value_name="pH")
%C df_before_melt; df_after_melt

            df_before_melt                       df_after_melt       
--------------------------------------     --------------------------
Contour  Depth  Depression  Slope  Top         Depth     Contour   pH
0         0-10         5.4    5.5  5.3     0    0-10  Depression  5.4
1        10-30         4.9    5.3  4.8     1   10-30  Depression  4.9
2        30-60         4.4    4.3  4.2     2   30-60  Depression  4.4
3        60-90         4.2    3.9  3.9     3   60-90  Depression  4.2
                                           4    0-10       Slope  5.5
                                           5   10-30       Slope  5.3
                                           6   30-60       Slope  4.3
                                           7   60-90       Slope  3.9
                                           8    0-10         Top  5.3
                                           9   10-30         Top  4.8
                                           10  30-60         Top  4.2
                    